In [ ]:
import pandas as pd
import re
import numpy as np# read csv data
import cx_Oracle

connection = None

username = 'XXXXXX'
password = 'XXXXXX'
dsn = '172.22.132.113/anonipac'
port = 1521
encoding = 'UTF-8'
service_name= 'anonipac'
table = 'PLN_HPC.PNL_TEXTO_ATENDIMENTO'

In [54]:
def processarTexto(texto):
    textoProcessado = texto.replace('#','')

    #trocando tracinho do começo por nada
    #textoProcessado =re.sub("^-{1,}" , "", textoProcessado)
    textoProcessado =re.sub("\n-{1,}" , "\n", textoProcessado)
    
    textoProcessado = textoProcessado.replace('--> ',' - ')
    textoProcessado = textoProcessado.replace('--','-')
    
    textoProcessado = textoProcessado.replace('\n\n','. ')
    textoProcessado = textoProcessado.replace('\n','; ')
    textoProcessado = textoProcessado.replace('*',' ')
    textoProcessado = textoProcessado.replace('<','')
    textoProcessado = textoProcessado.replace('>','')
    textoProcessado = textoProcessado.replace('  ',' ')
    textoProcessado = textoProcessado.replace('. .','.')
    textoProcessado = textoProcessado.replace(' .','.')
    textoProcessado = textoProcessado.replace('..','.')
    textoProcessado = textoProcessado.replace(':.',':')
    textoProcessado = textoProcessado.replace(':;',':')
    textoProcessado = textoProcessado.replace(': ;',':')
    textoProcessado = textoProcessado.replace('.;','.')
    textoProcessado = textoProcessado.replace('. ;','.')
    textoProcessado = textoProcessado.replace(': .',':')
    #textoProcessado = textoProcessado.replace('-',' ')
    
    textoProcessado = textoProcessado.replace('//',',')
    textoProcessado = textoProcessado.replace('  ',' ')
    textoProcessado = textoProcessado.replace(' ;',';')
    textoProcessado = textoProcessado.replace(' ,',';')
    
    #trocando dois ou mais espaços em branco por um só
    textoProcessado =re.sub("\s\s+" , " ", textoProcessado)

    return textoProcessado
    

In [55]:
try:
    connection = cx_Oracle.connect(
        username,
        password,
        dsn,
        encoding=encoding)

    cur = connection.cursor()
    # faz select de 50 em 50 até chegar em 160.000
    cur.execute("select * from PLN_HPC.PNL_TEXTO_ATENDIMENTO OFFSET 10 ROWS FETCH NEXT 10 ROWS ONLY;")
    res = cur.fetchall()
    
    for deptno, dname, loc in cur:
        print("campo1: ", deptno)
        print("campo2: ", dname)
        print("campo3:", loc)
        textoProcessado = processarTexto(texto)
        comando = 'UPDATE PLN_HPC.PNL_TEXTO_ATENDIMENTO set textoProcessado = :textoProcessado where id = :id'
        cur.execute(comando, [textoProcessado, str(id)])
        # commit the change
        connection.commit()
except cx_Oracle.Error as error:
    print(error)

finally:
    # release the connection
    if connection:
        connection.close()
